In [24]:
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np
import time

from ase_interface import ANIENS
from ase_interface import ensemblemolecule

from ase.atoms import Atoms
from ase.optimize import BFGS, LBFGS

import pyaniasetools as pya

import batch_optimizers as opt

import matplotlib.pyplot as plt

In [25]:
mol = hdt.readxyz2('/nh/nest/u/jsmith/Research/confs_test/capsaicin.xyz')
N = 50

wkdir = '/nh/nest/u/jsmith/Research/confs_test/arc13/'
cns = wkdir + 'rHCNO-4.6R_16-3.1A_a4-8.params'
sae = wkdir + 'sae_linfit.dat'
nnf = wkdir + 'train'
Nn = 5

#wkdir = '/nh/nest/u/jsmith/scratch/Gits/ANI-Networks/networks/ANI-1x/ANI-9.0.4_netarch8/'
#cns = wkdir + 'rHCNO-4.6A_16-3.1A_a4-8.params'
#sae = wkdir + 'sae_6-31gd.dat'
#nnf = wkdir + 'train'
#Nn = 1

In [26]:
Xl = np.stack([mol[0][0] for i in range(N)])
S = list(mol[1])
Xl.shape

(50, 49, 3)

In [27]:
X = Xl+np.random.uniform(-0.15,0.15,size=Xl.shape)
X = np.asarray(X, dtype=np.float32)
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify1.xyz',X,S)

In [28]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

In [33]:
from ase.parallel import world
from ase import parallel
print(parallel.size)

1


In [31]:
start_time = time.time()
for i,x in enumerate(X[0:1]):
    mol = Atoms(S,x)
    mol.set_calculator(ANIENS(aens))
    
    dyn = LBFGS(mol)
    dyn.run(fmax=0.00018)
    print('Molecule',i,'complete')
print('[ANI Total time:', time.time() - start_time, 'seconds]')

       Step     Time          Energy         fmax
LBFGS:    0 16:13:20   -26712.032989       19.0438
LBFGS:    1 16:13:20   -26718.580604       12.9861
LBFGS:    2 16:13:20   -26721.685241       10.5274
LBFGS:    3 16:13:20   -26724.546694        8.2632
LBFGS:    4 16:13:20   -26726.959204        5.5674
LBFGS:    5 16:13:20   -26728.756319        3.2401
LBFGS:    6 16:13:20   -26729.711037        1.2458
LBFGS:    7 16:13:20   -26730.098393        1.8587
LBFGS:    8 16:13:20   -26730.251369        2.4583
LBFGS:    9 16:13:20   -26730.408498        1.1447
LBFGS:   10 16:13:20   -26730.561638        1.3211
LBFGS:   11 16:13:20   -26730.608861        0.8586
LBFGS:   12 16:13:20   -26730.650988        0.3565
LBFGS:   13 16:13:20   -26730.696082        0.4490
LBFGS:   14 16:13:20   -26730.720471        0.4261
LBFGS:   15 16:13:20   -26730.754956        0.2913
LBFGS:   16 16:13:20   -26730.776877        0.2889
LBFGS:   17 16:13:20   -26730.796658        0.2696
LBFGS:   18 16:13:20   -26730.81

LBFGS:  160 16:13:21   -26730.976800        0.0090
LBFGS:  161 16:13:21   -26730.976823        0.0083
LBFGS:  162 16:13:21   -26730.976857        0.0084
LBFGS:  163 16:13:21   -26730.976885        0.0104
LBFGS:  164 16:13:21   -26730.976915        0.0089
LBFGS:  165 16:13:21   -26730.976930        0.0088
LBFGS:  166 16:13:21   -26730.976952        0.0071
LBFGS:  167 16:13:21   -26730.976966        0.0072
LBFGS:  168 16:13:21   -26730.976981        0.0090
LBFGS:  169 16:13:21   -26730.977004        0.0081
LBFGS:  170 16:13:21   -26730.977025        0.0083
LBFGS:  171 16:13:21   -26730.977040        0.0073
LBFGS:  172 16:13:21   -26730.977056        0.0069
LBFGS:  173 16:13:21   -26730.977068        0.0085
LBFGS:  174 16:13:22   -26730.977076        0.0074
LBFGS:  175 16:13:22   -26730.977095        0.0055
LBFGS:  176 16:13:22   -26730.977115        0.0057
LBFGS:  177 16:13:22   -26730.977126        0.0056
LBFGS:  178 16:13:22   -26730.977146        0.0064
LBFGS:  179 16:13:22   -26730.9

LBFGS:  321 16:13:23   -26730.979651        0.0010
LBFGS:  322 16:13:23   -26730.979653        0.0012
LBFGS:  323 16:13:23   -26730.979643        0.0012
LBFGS:  324 16:13:23   -26730.979648        0.0009
LBFGS:  325 16:13:23   -26730.979652        0.0007
LBFGS:  326 16:13:23   -26730.979651        0.0010
LBFGS:  327 16:13:23   -26730.979655        0.0015
LBFGS:  328 16:13:23   -26730.979641        0.0007
LBFGS:  329 16:13:23   -26730.979649        0.0008
LBFGS:  330 16:13:23   -26730.979662        0.0010
LBFGS:  331 16:13:23   -26730.979650        0.0009
LBFGS:  332 16:13:23   -26730.979650        0.0006
LBFGS:  333 16:13:23   -26730.979650        0.0009
LBFGS:  334 16:13:23   -26730.979651        0.0011
LBFGS:  335 16:13:23   -26730.979661        0.0014
LBFGS:  336 16:13:23   -26730.979650        0.0012
LBFGS:  337 16:13:23   -26730.979654        0.0009
LBFGS:  338 16:13:23   -26730.979654        0.0010
LBFGS:  339 16:13:23   -26730.979655        0.0012
LBFGS:  340 16:13:23   -26730.9

In [ ]:
# molopt = opt.moleculeOptimizer(cns, sae, nnf, Nn, 0)

In [ ]:
# start_time = time.time()
# X_it = molopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=1.0, printer=False)
# X_it = molopt.optimizeConjugateGradient(X_it,S,alpha=0.0001,convergence=0.005, maxsteps=5000, printstep=100, printer=False)
# print('[Complete Optimization Time:',time.time()-start_time,'s]')
# hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol1.xyz',np.array(X_it),S)

In [ ]:
cnfopt = opt.conformerOptimizer(cns, sae, nnf, Nn, 0)

In [ ]:
start_time = time.time()
X_it = cnfopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=10.0, maxsteps=10000, printstep=10)
X_it = cnfopt.optimizeConjugateGradient(X_it,S,alpha=0.0003,convergence=0.005, maxsteps=10000, printstep=10)
print('[Complete Optimization Time:',time.time()-start_time,'s]')
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol2.xyz',np.array(X_it),S)